# Перенос стиля. Подготовка данных 1.

В данном ноутбуке скачиваем стихи Пушкина и Цветаевой с сайта http://stih.su/.

In [4]:
from lxml import etree, html as lhtml
from bs4 import BeautifulSoup
import urllib3
import urllib.parse
import pandas as pd
import re
from tqdm import tqdm

In [7]:
import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

In [9]:
shop_page = "http://stih.su/"

In [10]:
authors = ['cvetaeva', 'pushkin']

In [11]:
def get_poems_urls(author_soup):
    poems_hrefs = []
    all_poems_h2 = author_soup.find_all('h2', class_="entry-title")

    for h2_poem in all_poems_h2:
        href_poem = h2_poem.find('a')['href']
        poems_hrefs.append(href_poem)
        
        
    if len(all_poems_h2) == 0:
        all_poems_h2 = author_soup.find('div', class_="years-category")
        all_hrefs = all_poems_h2.find_all('a')
        for href in all_hrefs:
            poem_href = href['href']
            if poem_href.startswith(shop_page):
                poems_hrefs.append(poem_href)
            else:
                poems_hrefs.append(shop_page+href['href'])
    return poems_hrefs

In [13]:
import re
def get_poem_text(poem_url):
    response = http.request('GET', poem_url)
    c=1
    while response.status != 200 and c<=5:
        response = http.request('GET', poem_url)
        c += 1
    if response.status != 200:
        print('Can\t load the page')
    poem_soup = BeautifulSoup(response.data, "lxml")
    poem_content = poem_soup.find('div', 'entry-content')

    poem_text = ''

    poems_p = poem_content.find_all('p')

    for poem_p in poems_p:
        if poem_p.text.lower().find("год написания") != -1 or re.search(r'18[0-9]+\sг.',poem_p.text) or re.search(r'\s19[0-9]+',poem_p.text):
            #print(poem_p.text)
            pass
        else:
            
            poem_text+= poem_p.text
            poem_text += '\n'

    poem_text = poem_text.lower()
    return poem_text

In [15]:
http = urllib3.PoolManager()

In [14]:
authors

['cvetaeva', 'pushkin']

In [16]:
poems = []
for author in authors:
    print(urllib.parse.urljoin(shop_page, author))
    cur_author_url = urllib.parse.urljoin(shop_page, author)
    response = http.request('GET', cur_author_url)
    c=1
    while response.status != 200 and c<=5:
        response = http.request('GET', cur_author_url)
        c += 1
    if response.status != 200:
        print('Can\t load the page')


    author_soup = BeautifulSoup(response.data, "lxml")
    poems_urls = get_poems_urls(author_soup)
    poem_texts = []
    for purl in poems_urls:
        poem_texts.append(get_poem_text(purl))
    sentences = tokenizer.tokenize(' '.join(poem_texts))
    poems.append('\n-------------\n\n'.join(poem_texts))
    print(author, len(poem_texts), len(sentences))

http://stih.su/cvetaeva


KeyboardInterrupt: 

In [131]:
for author, poem_text in zip(['cvetaeva'], poems):
    with open('data/'+author+'all.txt', 'w', encoding='UTF-8') as file:
        file.write(poem_text)
    file.close()

In [121]:
get_poem_text(purl)

1821 г.


'всё так же [ль] осеняют своды\n[сей храм] [парнасских] трех цариц?\nвсё те же ль клики юных жриц?\nвсё те же [ль] вьются хороводы?…\nужель умолк волшебный глас\nсеменовой, сей чудной музы?\nужель, навек оставя нас,\nона расторгла с фебом узы,\nи славы русской луч угас?\nне верю! вновь она восстанет.\nей вновь готова дань сердец,\nпред нами долго не [увянет]\nее торжественный венец.\nи для нее любовник [?] славы,\nнаперсник важных аонид [?],\nмладой катенин воскресит\nэсхила гений величавый\nи ей [порфиру] возвратит.\n'